In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
## LOding the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Preprocessing the dataset
###Drop irrelevant columns
data=df
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [4]:
label_encoder= LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
ohe = OneHotEncoder(handle_unknown='ignore')
geography_encoded = ohe.fit_transform(data[['Geography']])
geography_df = pd.DataFrame(geography_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography',axis=1), geography_df], axis=1)

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [11]:
#Split the dataset into training and testing sets
X= data.drop('EstimatedSalary',axis=1)
y= data['EstimatedSalary']

In [12]:
# Train-test split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.appl

In [13]:
#Save the preprocesser
with open('scalerregression.pkl', 'wb') as file:
    pickle.dump(scaler, file)

#Save the label encoder
with open('labelencoderregres.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)
#Save the one hot encoder
with open('ohe_regression.pkl', 'wb') as file:
    pickle.dump(ohe, file)

#ANN With Regression

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import datetime

In [15]:
from tensorflow.keras.models import Sequential #ANN
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [17]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),# Input layer with number of features and first hidden layer
    Dense(32,activation='relu'), # Second hidden layer
    Dense(16, activation='relu'),# Third hidden layer
    Dense(1) # Here linear activation is used for regression output
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
#optimizers 
from tensorflow.keras.optimizers import Adam

# loss function
from tensorflow.keras.losses import MeanAbsoluteError

Adam_optimizer = Adam(learning_rate=0.01)

loss_function = MeanAbsoluteError()

In [20]:
model.compile(optimizer=Adam_optimizer, loss=loss_function, metrics=['mae'])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard,CSVLogger
## Setting up TensorBoard
log_dir = "regression/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create a TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

#CSV logger
csv_logger = CSVLogger('regressiontraining_log.csv', append=True, separator=';')

In [22]:
#Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [23]:
### TRain the model
salarymodel = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100,
                    callbacks=[early_stopping, tensorboard_callback, csv_logger])

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 65652.4453 - mae: 65652.4453 - val_loss: 50390.0234 - val_mae: 50390.0234
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 49969.7383 - mae: 49969.7383 - val_loss: 50911.2031 - val_mae: 50911.2031
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 49766.9570 - mae: 49766.9570 - val_loss: 50256.6680 - val_mae: 50256.6680
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 49633.1875 - mae: 49633.1875 - val_loss: 50393.0273 - val_mae: 50393.0273
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 49572.2031 - mae: 49572.2031 - val_loss: 50886.5469 - val_mae: 50886.5469
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 49506.6641 - mae: 49506.6641 - val_loss: 50318.0781 - val_mae: 50318.0781
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 49476.6289 - mae: 49476.6289 

In [25]:
model.save('salary_model.h5')

c:\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
%tensorboard --logdir regression/logs/fit/20250704-143309
print("Model training completed and saved as 'churn_model.h5'.")
print(f"tensorboard running on port 6006 {http://localhost:6006}")

UsageError: Line magic function `%tensorboard` not found.


In [26]:
#Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 - 0s - loss: 50256.6680 - mae: 50256.6680 - 76ms/epoch - 1ms/step
Test Loss: 50256.66796875, Test MAE: 50256.66796875
